## Calculate Phi spatial average within a basin or within LMEs

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA']/1e10 #km*km
KMT = dset['KMT']
darea = darea*dz/dz

In [4]:
darea = darea.transpose("z_t","nlat","nlon")

In [5]:
nlat = dset.dims['nlat']
nlon = dset.dims['nlon']
for xx in range(nlat):
    for yy in range(nlon):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            darea[kk:, xx, yy] = float("nan")

In [6]:
filename = f'{path}/DPLE-FOSI_hindcast_Phi_1948-2017_monthly.nc'
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)

In [7]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']
lme = [1,2,3,4,5,6,7,8,9,10,65]

In [8]:
nmon = ds.dims['time']
nver = ds.dims['z_t']
dsf = xr.Dataset()
for ii in range(9):
#for ii in range(13):
    dsf[f'Phi_{ii}'] = ds[f'Phi_{ii}']*darea

In [9]:
phi_box = np.ma.zeros([9, len(lme), nmon, nver])
#phi_box = np.ma.zeros([13, len(lme), nmon, nver])
phi_box.mask = True
# select a LME
for box,nn in zip(lme,range(len(lme))):
    print(box)
    latloc, lonloc = np.where(lme_mask == box)
    ar1 = np.ma.zeros([len(latloc),nver]);ar1.mask=True
    ar2 = np.ma.zeros([9, len(latloc),nmon,nver]);ar2.mask=True
    for bb in range(len(latloc)):
        ar1[bb] = darea.isel(nlat=latloc[bb], nlon=lonloc[bb])
        for ii in range(9):
            ar2[ii,bb] = dsf[f'Phi_{ii}'].isel(nlat=latloc[bb], nlon=lonloc[bb])
    #
    tarea = np.nansum(ar1,axis=0)
    tarea = np.tile(tarea[np.newaxis,np.newaxis,:],[9,nmon,1])
    phi_box[:,nn] = np.nansum(ar2,axis=1)/tarea   

1
2
3
4
5
6
7
8
9
10
65


In [10]:
dsf = xr.Dataset()
for ii in range(9):
#for ii in range(13):
    dsf[f'Phi_{ii}'] = xr.DataArray(data=phi_box[ii], coords=dict(lme=('lme', lme), time=('time', ds.coords['time'].values), z_t=('z_t', ds.coords['z_t'].values)), dims=('lme','time','z_t'))

In [11]:
dsf.load()

<xarray.Dataset>
Dimensions:  (lme: 11, time: 840, z_t: 60)
Coordinates:
  * lme      (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
Data variables:
    Phi_0    (lme, time, z_t) float64 0.9192 0.9204 0.9217 ... 0.9315 nan nan
    Phi_1    (lme, time, z_t) float64 1.203 1.205 1.206 1.225 ... 1.25 nan nan
    Phi_2    (lme, time, z_t) float64 1.852 1.854 1.856 1.862 ... 1.999 nan nan
    Phi_3    (lme, time, z_t) float64 1.213 1.215 1.216 1.244 ... 1.229 nan nan
    Phi_4    (lme, time, z_t) float64 1.587 1.59 1.592 1.616 ... 1.65 nan nan
    Phi_5    (lme, time, z_t) float64 2.443 2.446 2.449 2.457 ... 2.637 nan nan
    Phi_6    (lme, time, z_t) float64 1.796 1.798 1.801 1.841 ... 1.82 nan nan
    Phi_7    (lme, time, z_t) float64 2.35 2.353 2.356 2.393 ... 2.443 nan nan
    Phi_8    (lme, time, z_t) float64 3.617 3.621 3.626 3.638 ... 3.904 nan nan

In [12]:
dsf.to_netcdf(f'{path}/DPLE-FOSI_hindcast_Phi_horizontal_mean_LME11_1948-2017_monthly.nc', mode='w')
#dsf.to_netcdf(f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_horizontal_mean_LME11_1948-2017_monthly.nc', mode='w')